### Setup
Download .sqlite file from https://www.kaggle.com/gabrio/board-games-dataset/data and save as database.sqlite

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sqlite3
conn = sqlite3.connect('database.sqlite')
conn.row_factory = sqlite3.Row
c = conn.cursor()

### Find games of mechanic

In [ ]:
FILTER_MECHANIC = '%Worker Placement%'

c.execute('SELECT * FROM BoardGames WHERE "attributes.boardgamemechanic" LIKE ?', [FILTER_MECHANIC])
rows = c.fetchall()
print('Found {} games'.format(len(rows)))

In [ ]:
c.execute('SELECT * FROM BoardGames WHERE "attributes.boardgamemechanic" LIKE ? AND "stats.subtype.boardgame.pos" IS NOT NULL', [FILTER_MECHANIC])
rows = c.fetchall()
print('Found {} ranked games'.format(len(rows)))

#r[stats.subtype.boardgame.pos] for r in rows

### Top Games

In [ ]:
TOP_N_LIMIT = 10

c.execute('SELECT * FROM BoardGames WHERE "attributes.boardgamemechanic" LIKE ? AND "stats.subtype.boardgame.pos" IS NOT NULL ORDER BY "stats.subtype.boardgame.pos" ASC LIMIT ?', [FILTER_MECHANIC, TOP_N_LIMIT])
rows = c.fetchall()

def create_games(rows):
    games = []
    for r in rows:
        game = {}
        for key in r.keys():
            game[key] = r[key]
        games.append(game)
    return games

games = pd.DataFrame(create_games(rows))
top_games = games[['stats.subtype.boardgame.pos', 'details.name', 'details.yearpublished', 'attributes.boardgamedesigner', 'stats.averageweight', 'details.minplayers', 'details.maxplayers', 'details.minplaytime', 'details.maxplaytime']]
top_games

### By Designer

In [ ]:
def calculate_designer_histogram(games):
    designers = [game.get('attributes.boardgamedesigner') for index, game in games.iterrows()]
    x = np.array(designers)
    unique, counts = np.unique(x, return_counts=True)
    sort = np.argsort(counts)
    counts = [counts[i] for i in sort]
    unique = [unique[i] for i in sort]
    hist = np.asarray((unique, counts))
    hist = np.flip(hist, axis=1)
    return hist

hist = calculate_designer_histogram(top_games)

In [ ]:
def show_designer_histogram(hist):
    plt.rcdefaults()
    fig, ax = plt.subplots()

    people = hist[0]
    y_pos = np.arange(len(people))
    performance = hist[1]

    ax.barh(y_pos, performance, align='center',
            color='green', ecolor='black')
    ax.set_yticks(y_pos)
    ax.set_yticklabels(people)
    ax.invert_yaxis()  # labels read top-to-bottom
    ax.set_ylabel('Designer')
    ax.set_xlabel('Games')
    ax.set_title('Leaderboard')
    plt.show()
    
show_designer_histogram(hist)

def find_games_by_designer(games, name):
    return games.where(games['attributes.boardgamedesigner'] == name).dropna(how='all')


for designer_name in hist[0]:
    designed_games = find_games_by_designer(games, designer_name)
    display(designed_games[['attributes.boardgamedesigner', 'stats.subtype.boardgame.pos', 'details.name', 'details.yearpublished', 'stats.averageweight', 'details.minplayers', 'details.maxplayers', 'details.minplaytime', 'details.maxplaytime']])
